# Overview

The `footystats` data set contains data related to the matches as sport events; most importantly, **goal timings**.  
The `Betfair` data set contains data related to the predictive markets associated with matches; most importantly, **odds (prices)**.  
The `footystats` data set contains only **relevant matches**, while the `Betfair` data set contains **all matches** with predictive markets for a given period of time.  
So, the natural way to merge both data sets is to find the `Betfair` `event id` for each match in the `footystats` data set.  
There is no common `key` between both data sets, so we use team names and the datetime of the match as a `key`.  

Objective:  
- find `Betfair` `event id` for relevant matches from `footystats` data set.  

Notes:
- team names may not be the same in both data sets. Use string similarity and check manually.

# Setup

## Imports

In [153]:
# data wrangling
import json
import numpy as np
import pandas as pd
# timezone
import pytz
# files, system
import os
# compressed files
import bz2
# processing time
from tqdm import tqdm
# string similarity
from fuzzywuzzy import fuzz
# regex
import re

# Get basic info for match identification (Betfair)

As we have ~300k Betfair match files and a maximum of ~40k are relevant for this research, we:  
1. get only basic information for match identification from the ~300k files;  
2. find Betfair `event id`s for relevant `footystats` matches;  
3. process Betfair data of relevant matches alone.

Most matches have one `.bz2` data file, but matches can have more than one file.  
For instance, the event `27853333` has files `1.125589876.bz2` and `1.125805946.bz2`.

In this section, we go over all Betfair data files, get basic information on each match and store the data.

In [2]:
BASE_FOLDER_PATH = r'..\data\raw\betfair'

Function to list all `.bz2` files in the Betfair raw data folder.

In [3]:
def list_files(file_path, file_type):
    """
    Walks through a file path and returns all files that have the specified file type.
    
    Params:
    file_path (str): The directory to search for files in and its subdirectories.
    file_type (str): The file extension to search for (e.g. '.txt', '.pdf').
    
    Returns:
    matching_files (list): A list of filenames that have the specified file type.
    """
    matching_files = []
    for dir_path, dir_names, file_names in os.walk(file_path):
        for filename in file_names:
            if filename.lower().endswith(file_type.lower()):
                matching_files.append(os.path.join(dir_path, filename))
    return matching_files

In [4]:
matching_files = list_files(BASE_FOLDER_PATH, '.bz2')

Example of file path.

In [5]:
matching_files[0]

'..\\data\\raw\\betfair\\xds_nfs\\edp_processed\\BASIC\\2018\\Dec\\1\\28997142\\1.150836575.bz2'

Betfair data files are structured as one JSON/dictionary per line.  
So we store the data of one file as a list of dictionaries.  
Below is a function to get a list of dictionaries from an original compressed `.bz2` data file.

In [6]:
def bz2_to_dict_list(bz2_file_path):
    """Converts a .bz2 file to a list of dictionaries.
    
    Args:
        bz2_file_path (str): Path to the .bz2 file.
    
    Returns:
        list of dict: List of dictionaries represented by the .bz2 file.
    """
    with bz2.BZ2File(bz2_file_path, "rb") as bz2_file:
        data = bz2_file.read().decode("utf-8")
        data_list = []
        for line in data.splitlines():
            data_list.append(json.loads(line))
    return(data_list)

To define how to get match identification information, we walk through an example to check the JSON structure.

In [7]:
bz2_file_path = '../data/raw/betfair/xds_nfs/edp_processed/BASIC/2021/Feb/4/30263011/1.178709515.bz2'

In [8]:
data_list = bz2_to_dict_list(bz2_file_path)

The first dictionary has the information that identifies the match.

In [9]:
data_list[0]

{'op': 'mcm',
 'clk': '2768636726',
 'pt': 1612139881671,
 'mc': [{'id': '1.178709515',
   'marketDefinition': {'bspMarket': False,
    'turnInPlayEnabled': True,
    'persistenceEnabled': True,
    'marketBaseRate': 5.0,
    'eventId': '30263011',
    'eventTypeId': '1',
    'numberOfWinners': 1,
    'bettingType': 'ODDS',
    'marketType': 'MATCH_ODDS',
    'marketTime': '2021-02-04T00:30:00.000Z',
    'suspendTime': '2021-02-04T00:30:00.000Z',
    'bspReconciled': False,
    'complete': True,
    'inPlay': False,
    'crossMatching': True,
    'runnersVoidable': False,
    'numberOfActiveRunners': 3,
    'betDelay': 0,
    'status': 'OPEN',
    'runners': [{'status': 'ACTIVE',
      'sortPriority': 1,
      'id': 198136,
      'name': 'Corinthians'},
     {'status': 'ACTIVE',
      'sortPriority': 2,
      'id': 5287019,
      'name': 'Ceara SC Fortaleza'},
     {'status': 'ACTIVE', 'sortPriority': 3, 'id': 58805, 'name': 'The Draw'}],
    'regulators': ['MR_INT'],
    'countryCode'

Example: home team name

In [10]:
data_list[0]['mc'][0]['marketDefinition']['runners'][0]['name']

'Corinthians'

In [11]:
data_list[0].keys()

dict_keys(['op', 'clk', 'pt', 'mc'])

After having explored the dictionary structure, we can write a function to get basic information for match identification.

In [12]:
EVENT_INFO_INDEX = 0
MC_INDEX = 0
HOME_TEAM_INDEX = 0
AWAY_TEAM_INDEX = 1

def get_event_basic_info(data):
    """
    Takes an event list of dictionaries and returns basic information for identification of the event.
    
    Params:
    data (list of dic): Event list of dictionaries.
    
    Returns:
    event_basic_info (dic): Basic information for identification of the event.
    """
    
    data_market_definition = data[EVENT_INFO_INDEX]['mc'][MC_INDEX]['marketDefinition']
    
    event_id = data_market_definition['eventId']
    country = data_market_definition['countryCode']
    open_date = data_market_definition['openDate']
    home_team = data_market_definition['runners'][HOME_TEAM_INDEX]['name']
    away_team = data_market_definition['runners'][AWAY_TEAM_INDEX]['name']
    event_name = data_market_definition['eventName']
    
    event_basic_info = {'event_id': event_id,
                        'country': country,
                        'open_date': open_date,
                        'home_team': home_team,
                        'away_team': away_team,
                        'event_name': event_name
                       }
    return event_basic_info
    

Function to get basic information for multiple files.

In [13]:
def get_event_basic_info_multiple(matching_files):
    """
    Get event basic info for multiple files as per list `matching_files`.
    Return dictionary {'file_path': event_basic_info} and list of files with reading errors.
        event_basic_info is a dictionary with basic info returned by get_event_basic_info().
    """
    event_basic_info_dic = {}
    errors_event_basic_info = {}

    for bz2_file_path in tqdm(matching_files):
        try:
            event_basic_info_dic[bz2_file_path] = get_event_basic_info(bz2_to_dict_list(bz2_file_path))
        except Exception as e:
            errors_event_basic_info[bz2_file_path] = e
    return event_basic_info_dic, errors_event_basic_info

Run for all files.

In [14]:
event_basic_info_dic, errors_event_basic_info = get_event_basic_info_multiple(matching_files)

100%|█████████████████████████████████████████████████████████████████████████| 303894/303894 [22:17<00:00, 227.13it/s]


A very small number of files could not be read or had formatting issues.

In [15]:
len(list(errors_event_basic_info.keys())) / len(list(event_basic_info_dic.keys()))

0.0004905430542066536

So, basic information for match identification in the Betfair data set is stored in `event_basic_info_dic`.  
Let us write it into a csv file for later reference.

In [16]:
EVENT_BASIC_INFO_DIC_PATH = '../data/interim/event_basic_info_dic.json'

In [593]:
with open(EVENT_BASIC_INFO_DIC_PATH, "w") as outfile:
    json.dump(event_basic_info_dic, outfile)

In case we need to load `event_basic_info_dic` from the file, the following code may be used:

In [594]:
with open(file=EVENT_BASIC_INFO_DIC_PATH, mode="r") as f:
    event_basic_info_dic = json.load(f)

# Find Betfair `event id`s for relevant footystats matches

Team names in Betfair and footystats may be different (e.g. Man Utd and Manchester United).  
So, we cannot directly build `keys` using team names and match datetimes.  
For each team name in the footystats data set, we define a list of compatible team names that appear in the Betfair data set.  
The lists of team names are obtained using `fuzzy` string similarity and by checking manually.  
Then, for each footystats record, we search all combinations of compatible Betfair home and away team names.  
For example:  
- footystats: `Manchester United x Manchester City, Mar 8 2020 - 4:30pm`.
- Betfair:
    - there are records of 'Man United' and 'Man Utd', but not 'Manchester United'.  
    - there are records of 'Manchester City' and 'Man City'.  
- Thus, we search for all 4 combinations of home and away team names, at the given match datetime.  

We explicitly search within the same country, as there are teams with the same name across countries (e.g. Arsenal-AR, Arsenal-GB).  
The steps are:
- get Betfair country-team_name pairs  
- get footystats country-team_name pairs  
- for each footystats country-team_name pair, get `fuzzy` most similar Betfair team names  
- perform manual checking 
- search `event id`s

## get Betfair country-team_name pairs

In [19]:
event_basic_info_df = pd.DataFrame(event_basic_info_dic.values())
event_basic_info_df['open_date'] = pd.to_datetime(event_basic_info_df['open_date'])
# the same match(event) may be divided into more than one data file, so remove duplicates
event_basic_info_df = event_basic_info_df.drop_duplicates()

In [20]:
event_basic_info_df

,event_id,country,open_date,home_team,away_team,event_name
0,28997142,FR,2018-12-01 16:00:00+00:00,Lille,Lyon,Lille v Lyon
1,28997143,FR,2018-12-01 19:00:00+00:00,Nimes,Amiens,Nimes v Amiens
2,28997145,FR,2018-12-01 19:00:00+00:00,Angers,Caen,Angers v Caen
3,28997146,FR,2018-12-01 19:00:00+00:00,Guingamp,Nice,Guingamp v Nice
4,28997147,FR,2018-12-01 19:00:00+00:00,Monaco,Montpellier,Monaco v Montpellier
...,...,...,...,...,...,...
303740,28893785,DE,2018-09-09 12:00:00+00:00,Eintracht Norderstedt,Luneburg,Eintracht Norderstedt v Luneburg
303741,28893786,DE,2018-09-09 13:00:00+00:00,Kieler SV Holstein II,Weiche Flensburg,Kieler SV Holstein II v Weiche Flensburg
303742,28893787,DE,2018-09-09 13:00:00+00:00,Wuppertaler,Alemannia Aachen,Wuppertaler v Alemannia Aachen
303743,28893799,GB,2018-09-09 13:00:00+00:00,South Sudan,Mali,South Sudan v Mali


Let us check the distribution of column `country`.

In [21]:
event_basic_info_df['country'].value_counts()

GB    61979
ES    41274
BR    28395
DE    24077
IT    19284
AR    16556
FR    12270
PT    11415
IL    10177
JP     9934
RU     9532
TR     7975
NL     7801
US     7333
CN     2729
AU     1594
CA      709
RO        3
BY        2
CM        2
AM        2
AO        2
AG        2
UZ        2
UY        1
NO        1
AD        1
VE        1
BZ        1
AW        1
IS        1
IE        1
TN        1
BA        1
BT        1
AT        1
EC        1
MA        1
Name: country, dtype: int64

There is a very small number of events in countries other than the list of relevant countries.  
Let us enforce the list of countries.

In [22]:
COUNTRY_CODES_PATH = '../betfair-api/configuration/countryCodes.csv'
country_codes = list(pd.read_csv(COUNTRY_CODES_PATH)['marketDefinition.countryCode'].values)
event_basic_info_df = event_basic_info_df[event_basic_info_df['country'].isin(country_codes)].copy()

In [23]:
event_basic_info_df

,event_id,country,open_date,home_team,away_team,event_name
0,28997142,FR,2018-12-01 16:00:00+00:00,Lille,Lyon,Lille v Lyon
1,28997143,FR,2018-12-01 19:00:00+00:00,Nimes,Amiens,Nimes v Amiens
2,28997145,FR,2018-12-01 19:00:00+00:00,Angers,Caen,Angers v Caen
3,28997146,FR,2018-12-01 19:00:00+00:00,Guingamp,Nice,Guingamp v Nice
4,28997147,FR,2018-12-01 19:00:00+00:00,Monaco,Montpellier,Monaco v Montpellier
...,...,...,...,...,...,...
303740,28893785,DE,2018-09-09 12:00:00+00:00,Eintracht Norderstedt,Luneburg,Eintracht Norderstedt v Luneburg
303741,28893786,DE,2018-09-09 13:00:00+00:00,Kieler SV Holstein II,Weiche Flensburg,Kieler SV Holstein II v Weiche Flensburg
303742,28893787,DE,2018-09-09 13:00:00+00:00,Wuppertaler,Alemannia Aachen,Wuppertaler v Alemannia Aachen
303743,28893799,GB,2018-09-09 13:00:00+00:00,South Sudan,Mali,South Sudan v Mali


Betfair records the country where a match took place, whereas footystats' `country` is the country of the *league*.  
So, matches played in Canada for the Major League Soccer (US) will be recorded as `CA` in Betfair and `US` in footystats.  
In the opposite direction, Betfair records as `GB` all matches played in Great Britain (e.g. English, Scottish leagues).  
We deal with `GB` later on, when we process footystats data.  
Now, get a column in which matches played in Canada show up as `US`.

In [24]:
event_basic_info_df['country_caus'] = event_basic_info_df['country'].replace('CA', 'US')

In [25]:
event_basic_info_df

,event_id,country,open_date,home_team,away_team,event_name,country_caus
0,28997142,FR,2018-12-01 16:00:00+00:00,Lille,Lyon,Lille v Lyon,FR
1,28997143,FR,2018-12-01 19:00:00+00:00,Nimes,Amiens,Nimes v Amiens,FR
2,28997145,FR,2018-12-01 19:00:00+00:00,Angers,Caen,Angers v Caen,FR
3,28997146,FR,2018-12-01 19:00:00+00:00,Guingamp,Nice,Guingamp v Nice,FR
4,28997147,FR,2018-12-01 19:00:00+00:00,Monaco,Montpellier,Monaco v Montpellier,FR
...,...,...,...,...,...,...,...
303740,28893785,DE,2018-09-09 12:00:00+00:00,Eintracht Norderstedt,Luneburg,Eintracht Norderstedt v Luneburg,DE
303741,28893786,DE,2018-09-09 13:00:00+00:00,Kieler SV Holstein II,Weiche Flensburg,Kieler SV Holstein II v Weiche Flensburg,DE
303742,28893787,DE,2018-09-09 13:00:00+00:00,Wuppertaler,Alemannia Aachen,Wuppertaler v Alemannia Aachen,DE
303743,28893799,GB,2018-09-09 13:00:00+00:00,South Sudan,Mali,South Sudan v Mali,GB


We now exclude matches that are obviously not relevant to this research, such as womens' matches, under 23, 'young', 'reserves'.

In [26]:
check_strings = [' (W)', 'U19', 'U20', 'U21', 'U23', ' (Y)', ' (Res)']
escaped_check_strings = [re.escape(s) for s in check_strings]
mask = ~event_basic_info_df['event_name'].str.contains('|'.join(escaped_check_strings), regex=True)
event_basic_info_df = event_basic_info_df[mask]
event_basic_info_df

,event_id,country,open_date,home_team,away_team,event_name,country_caus
0,28997142,FR,2018-12-01 16:00:00+00:00,Lille,Lyon,Lille v Lyon,FR
1,28997143,FR,2018-12-01 19:00:00+00:00,Nimes,Amiens,Nimes v Amiens,FR
2,28997145,FR,2018-12-01 19:00:00+00:00,Angers,Caen,Angers v Caen,FR
3,28997146,FR,2018-12-01 19:00:00+00:00,Guingamp,Nice,Guingamp v Nice,FR
4,28997147,FR,2018-12-01 19:00:00+00:00,Monaco,Montpellier,Monaco v Montpellier,FR
...,...,...,...,...,...,...,...
303740,28893785,DE,2018-09-09 12:00:00+00:00,Eintracht Norderstedt,Luneburg,Eintracht Norderstedt v Luneburg,DE
303741,28893786,DE,2018-09-09 13:00:00+00:00,Kieler SV Holstein II,Weiche Flensburg,Kieler SV Holstein II v Weiche Flensburg,DE
303742,28893787,DE,2018-09-09 13:00:00+00:00,Wuppertaler,Alemannia Aachen,Wuppertaler v Alemannia Aachen,DE
303743,28893799,GB,2018-09-09 13:00:00+00:00,South Sudan,Mali,South Sudan v Mali,GB


We can now get team names.

In [47]:
bf_home_team_names = event_basic_info_df[['country_caus', 'home_team']].rename(columns={'home_team': 'team_name'})
bf_away_team_names = event_basic_info_df[['country_caus', 'away_team']].rename(columns={'away_team': 'team_name'})
bf_team_names = pd.concat([bf_home_team_names, bf_away_team_names]).drop_duplicates().reset_index().drop(columns=['index'])
bf_team_names

,country_caus,team_name
0,FR,Lille
1,FR,Nimes
2,FR,Angers
3,FR,Guingamp
4,FR,Monaco
...,...,...
12661,DE,Peru
12662,DE,Eintracht Braunschweig
12663,DE,Heeslinger
12664,JP,Sfida Setagaya


Export

In [48]:
BF_TEAM_NAMES_PATH = '../data/interim/bf_team_names.csv'

In [49]:
bf_team_names.to_csv(BF_TEAM_NAMES_PATH, index=False)

## get footystats country-team_name pairs

In [50]:
FOOTYSTATS_FILEPATH = '../data/interim/footy.csv'

In [51]:
footy = pd.read_csv(FOOTYSTATS_FILEPATH, parse_dates=['date_GMT'])

In [52]:
footy

,timestamp,date_GMT,status,home_team_name,away_team_name,home_team_goal_count,away_team_goal_count,home_team_goal_timings,away_team_goal_timings,country
0,1430589600,2015-05-02 18:00:00,complete,Olimpo,Estudiantes,0,0,NaN,NaN,argentina
1,1430597700,2015-05-02 20:15:00,complete,Gimnasia La Plata,Newell's Old Boys,0,0,NaN,NaN,argentina
2,1430601000,2015-05-02 21:10:00,complete,San Lorenzo,Vélez Sarsfield,1,0,83,NaN,argentina
3,1430605800,2015-05-02 22:30:00,complete,Argentinos Juniors,Aldosivi,0,1,NaN,45'1,argentina
4,1430608500,2015-05-02 23:15:00,complete,Racing Club,Lanús,2,0,"72,90",NaN,argentina
...,...,...,...,...,...,...,...,...,...,...
36081,1583632800,2020-03-08 02:00:00,complete,Colorado Rapids,Orlando City,2,1,"64,90",82,usa
36082,1583636400,2020-03-08 03:00:00,complete,LA Galaxy,Vancouver Whitecaps,0,1,NaN,74,usa
36083,1583636400,2020-03-08 03:00:00,complete,Seattle Sounders,Columbus Crew,1,1,79,33,usa
36084,1583708400,2020-03-08 23:00:00,complete,Portland Timbers,Nashville SC,1,0,12,NaN,usa


Get country codes, taking into consideration that English and Scottish leagues are recorded as `GB` in Betfair.

In [53]:
FOOTY_COUNTRY_CODES_PATH = 'configuration/footy_country_codes.csv'
footy_country_codes = pd.read_csv(FOOTY_COUNTRY_CODES_PATH)
footy_country_codes

,country_name,country_code
0,argentina,AR
1,australia,AU
2,brazil,BR
3,china,CN
4,england,GB
5,france,FR
6,germany,DE
7,israel,IL
8,italy,IT
9,japan,JP


In [54]:
footy = footy.merge(footy_country_codes, 
                    how='left', 
                    left_on='country', 
                    right_on='country_name').drop(columns=['country_name'])

In [55]:
footy

,timestamp,date_GMT,status,home_team_name,away_team_name,home_team_goal_count,away_team_goal_count,home_team_goal_timings,away_team_goal_timings,country,country_code
0,1430589600,2015-05-02 18:00:00,complete,Olimpo,Estudiantes,0,0,NaN,NaN,argentina,AR
1,1430597700,2015-05-02 20:15:00,complete,Gimnasia La Plata,Newell's Old Boys,0,0,NaN,NaN,argentina,AR
2,1430601000,2015-05-02 21:10:00,complete,San Lorenzo,Vélez Sarsfield,1,0,83,NaN,argentina,AR
3,1430605800,2015-05-02 22:30:00,complete,Argentinos Juniors,Aldosivi,0,1,NaN,45'1,argentina,AR
4,1430608500,2015-05-02 23:15:00,complete,Racing Club,Lanús,2,0,"72,90",NaN,argentina,AR
...,...,...,...,...,...,...,...,...,...,...,...
36081,1583632800,2020-03-08 02:00:00,complete,Colorado Rapids,Orlando City,2,1,"64,90",82,usa,US
36082,1583636400,2020-03-08 03:00:00,complete,LA Galaxy,Vancouver Whitecaps,0,1,NaN,74,usa,US
36083,1583636400,2020-03-08 03:00:00,complete,Seattle Sounders,Columbus Crew,1,1,79,33,usa,US
36084,1583708400,2020-03-08 23:00:00,complete,Portland Timbers,Nashville SC,1,0,12,NaN,usa,US


In [56]:
footy_home_team_names = footy[['country_code', 'home_team_name']].rename(columns={'home_team_name': 'team_name'})
footy_away_team_names = footy[['country_code', 'away_team_name']].rename(columns={'away_team_name': 'team_name'})
footy_team_names = pd.concat([footy_home_team_names, footy_away_team_names]).drop_duplicates().reset_index().drop(columns=['index'])
footy_team_names = footy_team_names.sort_values(['country_code', 'team_name'])
footy_team_names

,country_code,team_name
21,AR,Aldosivi
3,AR,Argentinos Juniors
11,AR,Arsenal
6,AR,Atlético Rafaela
31,AR,Atlético Tucumán
...,...,...
530,US,SJ Earthquakes
531,US,Seattle Sounders
519,US,Sporting KC
528,US,Toronto


In [57]:
FOOTY_TEAM_NAMES_PATH = '../data/interim/footy_team_names.csv'

In [58]:
footy_team_names.to_csv(FOOTY_TEAM_NAMES_PATH, index=False)

## get `fuzzy` most similar Betfair team names

The following function takes the DataFrames with countries and team names that we got in the previous sections, and returns the `n` most similar Betfair team names and scores for each footystats team name.

In [61]:
def get_similar_teams(footy_team_names, bf_team_names, n=3):
    """
    Given two pandas DataFrames with countries and team names, return the n most similar team names from the second list 
    in the same country, for each team name in the first list, along with the string similarity score.
    
    :param footy_team_names: DataFrame of countries and team names from footystats
    :param bf_team_names: DataFrame of countries and team names from Betfair
    :param n: the number of most similar teams to return
    :return: a dictionary mapping each pair country-footy team name to a list of n tuples, 
            where each tuple contains a team name from betfair_team_names and its similarity score to the footy team name
    """
    result = {}
    for country_code, footy_team in tqdm(zip(footy_team_names['country_code'], footy_team_names['team_name'])):
        scores = []
        for betfair_team in bf_team_names[bf_team_names['country_caus'] == country_code]['team_name']:
            scores.append((betfair_team, fuzz.ratio(footy_team, betfair_team)))
        scores = sorted(scores, key=lambda x: x[1], reverse=True)[:n]
        result[(country_code, footy_team)] = scores
    return result

We run the function with `n=13`. This is not a 'magic number', but the result of an iteractive process with the next acticity, the manual checking step.  
For most teams, the first or second most similar team names suffice, but we surprisingly get relevant names at lower ranks for a non-negligible minority of teams.

In [62]:
similar_team_names = get_similar_teams(footy_team_names, bf_team_names, n=13)

539it [00:07, 68.32it/s] 


In [63]:
footy_team_names_scores = footy_team_names.copy()
footy_team_names_scores['scores'] = [similar_team_names[(country_code, footy_team)] 
                                     for (country_code, footy_team)
                                     in zip(footy_team_names['country_code'], footy_team_names['team_name'])]
footy_team_names_scores

,country_code,team_name,scores
21,AR,Aldosivi,"[(Aldosivi, 100), (Altos, 62), (CA Aldosivi Re..."
3,AR,Argentinos Juniors,"[(Argentinos Juniors, 100), (Argentinos Junior..."
11,AR,Arsenal,"[(Arsenal FC, 82), (Arenas, 77), (Argentina, 6..."
6,AR,Atlético Rafaela,"[(Atletico Rafaela, 94), (Atletico Parana, 71)..."
31,AR,Atlético Tucumán,"[(Atletico Tucuman, 88), (Club Atletico Tucuma..."
...,...,...,...
530,US,SJ Earthquakes,"[(San Jose Earthquakes, 82), (San Jose Earthqu..."
531,US,Seattle Sounders,"[(Seattle Sounders, 100), (Seattle Sounders 2,..."
519,US,Sporting KC,"[(Sporting Kansas, 77), (Sporting Orlando, 67)..."
528,US,Toronto,"[(Toronto FC, 82), (Toronto FC II, 70), (Toron..."


Explode the `scores` column to prepare a DataFrame to be used in the manual checking step.

In [64]:
footy_team_names_scores_exploded = pd.concat([footy_team_names_scores, 
                                              pd.concat([footy_team_names_scores['scores'].apply(pd.Series)[col].apply(pd.Series) for col in range(0, 13)], 
                                                        axis=1)], 
                                             axis=1).drop(columns=['scores'])
footy_team_names_scores_exploded

,country_code,team_name,0,1,0,1,0,1,0,1,...,0,1,0,1,0,1,0,1,0,1
21,AR,Aldosivi,Aldosivi,100,Altos,62,CA Aldosivi Reserves,57,Bolivia,53,...,Arg de Rosario,45,CA Florida,44,El Porvenir,42,Almagro,40,Rosario,40
3,AR,Argentinos Juniors,Argentinos Juniors,100,Argentinos Juniors Res,90,Argentinos Jrs,88,Argentinos Juniors Reserves,80,...,Argentino Merlo,67,Argentino,67,Argentino MM,67,Argentino Pergamino,65,Argentino Monte Maiz,63
11,AR,Arsenal,Arsenal FC,82,Arenas,77,Argentina,62,Arsenal FC (Arg),61,...,CA Penaflor,56,CA Regional,56,CA Barracas Central,54,Ros. Central,53,Barracas Central,52
6,AR,Atlético Rafaela,Atletico Rafaela,94,Atletico Parana,71,Atletico Campana,69,Atletico Raci,69,...,Atletico Cuyaya,65,Atletico Marquesado,63,CA 9 de Julio (Rafaela),62,Club Atletico Parana,61,Rafaela,61
31,AR,Atlético Tucumán,Atletico Tucuman,88,Club Atletico Tucuman,76,Atl Tucuman,74,Atletico Campana,62,...,Atletico MG,59,Atletico Parana,58,Atletico Cuyaya,58,Atletico Camioneros,57,Atletico Concepcion,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,US,SJ Earthquakes,San Jose Earthquakes,82,San Jose Earthquakes II,76,Martinique,50,Schalke,48,...,Johor Darul Takzim,44,Santa Cruz Breakers FC,44,Santa Cruz Breaker,44,Fort Worth Vaqueros FC,44,Real Salt Lake,43
531,US,Seattle Sounders,Seattle Sounders,100,Seattle Sounders 2,94,Seattle Sounders II,91,Seattle Sounders FC II,84,...,OSA Seattle FC,53,Golden State Misioneros,51,Real Monarchs SL,50,Oakland Stompers,50,Western Mass Pioneers,49
519,US,Sporting KC,Sporting Kansas,77,Sporting Orlando,67,Sporting Kansas City II,65,Khalsa Sporting Club,65,...,Wayne County Sporting,50,Austin FC,50,Fort Wayne FC,50,Boston FC,50,Floridians FC,50
528,US,Toronto,Toronto FC,82,Toronto FC II,70,Toronto Croatia,64,Tormenta 2,59,...,Burlington,47,Cotonsport,47,Fiorentina,47,Dragon,46,El Farolito,44


In [65]:
# get column names
cols = ['country_code', 'team_name']
for i in range(13):
    cols.append(f'bf_team{i+1}')
    cols.append(f'score{i+1}')

In [66]:
footy_team_names_scores_exploded.columns = cols
footy_team_names_scores_exploded

,country_code,team_name,bf_team1,score1,bf_team2,score2,bf_team3,score3,bf_team4,score4,...,bf_team9,score9,bf_team10,score10,bf_team11,score11,bf_team12,score12,bf_team13,score13
21,AR,Aldosivi,Aldosivi,100,Altos,62,CA Aldosivi Reserves,57,Bolivia,53,...,Arg de Rosario,45,CA Florida,44,El Porvenir,42,Almagro,40,Rosario,40
3,AR,Argentinos Juniors,Argentinos Juniors,100,Argentinos Juniors Res,90,Argentinos Jrs,88,Argentinos Juniors Reserves,80,...,Argentino Merlo,67,Argentino,67,Argentino MM,67,Argentino Pergamino,65,Argentino Monte Maiz,63
11,AR,Arsenal,Arsenal FC,82,Arenas,77,Argentina,62,Arsenal FC (Arg),61,...,CA Penaflor,56,CA Regional,56,CA Barracas Central,54,Ros. Central,53,Barracas Central,52
6,AR,Atlético Rafaela,Atletico Rafaela,94,Atletico Parana,71,Atletico Campana,69,Atletico Raci,69,...,Atletico Cuyaya,65,Atletico Marquesado,63,CA 9 de Julio (Rafaela),62,Club Atletico Parana,61,Rafaela,61
31,AR,Atlético Tucumán,Atletico Tucuman,88,Club Atletico Tucuman,76,Atl Tucuman,74,Atletico Campana,62,...,Atletico MG,59,Atletico Parana,58,Atletico Cuyaya,58,Atletico Camioneros,57,Atletico Concepcion,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,US,SJ Earthquakes,San Jose Earthquakes,82,San Jose Earthquakes II,76,Martinique,50,Schalke,48,...,Johor Darul Takzim,44,Santa Cruz Breakers FC,44,Santa Cruz Breaker,44,Fort Worth Vaqueros FC,44,Real Salt Lake,43
531,US,Seattle Sounders,Seattle Sounders,100,Seattle Sounders 2,94,Seattle Sounders II,91,Seattle Sounders FC II,84,...,OSA Seattle FC,53,Golden State Misioneros,51,Real Monarchs SL,50,Oakland Stompers,50,Western Mass Pioneers,49
519,US,Sporting KC,Sporting Kansas,77,Sporting Orlando,67,Sporting Kansas City II,65,Khalsa Sporting Club,65,...,Wayne County Sporting,50,Austin FC,50,Fort Wayne FC,50,Boston FC,50,Floridians FC,50
528,US,Toronto,Toronto FC,82,Toronto FC II,70,Toronto Croatia,64,Tormenta 2,59,...,Burlington,47,Cotonsport,47,Fiorentina,47,Dragon,46,El Farolito,44


The DataFrame is exported to be used in the manual checking step.

In [67]:
FOOTY_TEAM_NAMES_SCORES_EXPLODED_PATH = '../data/interim/footy_team_names_scores_exploded.csv'

In [68]:
footy_team_names_scores_exploded.to_csv(FOOTY_TEAM_NAMES_SCORES_EXPLODED_PATH, index=False)

## manual checking

The team names were checked manually and the mapping of footystats team names to Betfair team names is stored in the file `team_names_manual_check.csv`.  
The process was not performed at once.  
Feedback from the number of matches with Betfair data per team was used to identify teams with former names or aliases.  

In [524]:
TEAM_NAMES_MANUAL_CHECK_PATH = '../data/interim/team_names_manual_check.csv'

In [525]:
team_names_manual_check = pd.read_csv(TEAM_NAMES_MANUAL_CHECK_PATH).drop(columns=['Column1'])
team_names_manual_check

,row_type,country_code,team_name,bf_team1,score1,bf_team2,score2,bf_team3,score3,bf_team4,...,score9,bf_team10,score10,bf_team11,score11,bf_team12,score12,bf_team13,score13,manual_insertion
0,teams,AR,Aldosivi,Aldosivi,100,Altos,62,CA Aldosivi Reserves,57,Bolivia,...,45,CA Florida,44,El Porvenir,42,Almagro,40,Rosario,40,NaN
1,zero_one,AR,Aldosivi,1,100,0,62,0,57,0,...,45,0,44,0,42,0,40,0,40,0
2,teams,AR,Argentinos Juniors,Argentinos Juniors,100,Argentinos Juniors Res,90,Argentinos Jrs,88,Argentinos Juniors Reserves,...,67,Argentino,67,Argentino MM,67,Argentino Pergamino,65,Argentino Monte Maiz,63,NaN
3,zero_one,AR,Argentinos Juniors,1,100,0,90,1,88,0,...,67,0,67,0,67,0,65,0,63,0
4,teams,AR,Arsenal,Arsenal FC,82,Arenas,77,Argentina,62,Arsenal FC (Arg),...,56,CA Regional,56,CA Barracas Central,54,Ros. Central,53,Barracas Central,52,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,zero_one,US,Sporting KC,1,77,0,67,0,65,0,...,50,0,50,0,50,0,50,0,50,1
1074,teams,US,Toronto,Toronto FC,82,Toronto FC II,70,Toronto Croatia,64,Tormenta 2,...,47,Cotonsport,47,Fiorentina,47,Dragon,46,El Farolito,44,NaN
1075,zero_one,US,Toronto,1,82,0,70,0,64,0,...,47,0,47,0,47,0,46,0,44,0
1076,teams,US,Vancouver Whitecaps,Vancouver Whitecaps,100,Vancouver Whitecaps II,93,Vancouver Utd,69,Vancouver Thunderbirds,...,43,Seacoast United Phantoms,42,Lane United FC,42,San Jose Earthquakes,41,Southern C Seahorses,41,NaN


Row types are alternate. For columns `bf_team{i}`, `teams` rows contain team names, and `zero_one` rows contain Booleans representing the relevance of `bf_team{i}` for the respective footystats `team_name`. 
This structure was convenient for manual checking, but we separate them here so we can work programatically.

In [526]:
teams = team_names_manual_check[team_names_manual_check['row_type'] == 'teams']
zero_one = team_names_manual_check[team_names_manual_check['row_type'] == 'zero_one']

In [527]:
# columns with bf team names
bf_team_cols = [f'bf_team{i+1}' for i in range(13)]
bf_team_cols.append('manual_insertion')

team_names_mapping = {}

# get list of relevant bf team names per footy team
for i, (country_code, team_name) in enumerate(zip(teams['country_code'], teams['team_name'])):
    team_names_mapping[(country_code, team_name)] = list(np.array(teams[bf_team_cols])[i, :][np.array(zero_one[bf_team_cols]).astype(int).astype(bool)[i, :]])

team_names_mapping: {(country, footy_team_name): [bf_team_name1, bf_team_name2, ...]}

In [528]:
team_names_mapping

{('AR', 'Aldosivi'): ['Aldosivi'],
 ('AR', 'Argentinos Juniors'): ['Argentinos Juniors',
  'Argentinos Jrs',
  'Arg Juniors'],
 ('AR', 'Arsenal'): ['Arsenal FC', 'Arsenal FC (Arg)', 'Arsenal De Sarandi'],
 ('AR', 'Atlético Rafaela'): ['Atletico Rafaela', 'Rafaela'],
 ('AR', 'Atlético Tucumán'): ['Atletico Tucuman',
  'Club Atletico Tucuman',
  'Atl Tucuman'],
 ('AR', 'Banfield'): ['Banfield'],
 ('AR', 'Belgrano'): ['Belgrano'],
 ('AR', 'Boca Juniors'): ['Boca Juniors'],
 ('AR', 'Central Córdoba SdE'): ['Central Cordoba (SdE)',
  'Central Cordoba',
  'CA Central Cordoba',
  'Cordoba (SdE)',
  'C Cordoba'],
 ('AR', 'Chacarita Juniors'): ['Chacarita'],
 ('AR', 'Colón'): ['Colon'],
 ('AR', 'Crucero del Norte'): ['Crucero del Norte',
  'Crucero del N',
  'CM Crucero del N',
  'Club Mutual Crucero del Norte',
  'CMC del Norte',
  'Club Mutual Crucero del Nor',
  'Club Mutual Crucero del No',
  'Club Mutual Crucero del ',
  'Crucero',
  'Club Mutual Crucero del'],
 ('AR', 'Defensa y Justicia'

Export

In [529]:
TEAM_NAMES_MAPPING_PATH = '../data/interim/team_names_mapping.json'

In [530]:
# JSON keys must be str, int, float, bool or None, not tuple.
# so we use the string representation of the tuples.
team_names_mapping_tuples_as_strings = {str(k): v for k, v in team_names_mapping.items()}

with open(TEAM_NAMES_MAPPING_PATH, "w") as outfile:
    json.dump(team_names_mapping_tuples_as_strings, outfile)

## search `event id`s

Now we can get Betfair `event id`s for the relevant matches in the footystats data set.  
So we:  
 - iterate over the `footy` DataFrame;
 - filter the row(s) in the Betfair `event_basic_info_df` DataFrame that have:
     - the same country as the instance in the `footy` DataFrame;  
     - compatible home and away team names;
     - the same match start datetime.

Get a list of lists `event_ids`: each element is a list with the Betfair `event id`(s) found for each match.

In [533]:
event_ids = []

for i in tqdm(range(len(footy))):
    mask_country = event_basic_info_df['country_caus'] == footy.iloc[i]['country_code']
    # bf datetime is TZ aware and footy datetime is not. So explicitly set `.tz_localize(pytz.UTC)`
    mask_datetime = event_basic_info_df['open_date'] == footy.iloc[i]['date_GMT'].tz_localize(pytz.UTC)
    mask_home = event_basic_info_df['home_team'].isin(team_names_mapping[(footy.iloc[i]['country_code'], footy.iloc[i]['home_team_name'])])
    mask_away = event_basic_info_df['away_team'].isin(team_names_mapping[(footy.iloc[i]['country_code'], footy.iloc[i]['away_team_name'])])
    mask = mask_country * mask_datetime * mask_home * mask_away
    event_ids.append(list(event_basic_info_df[mask]['event_id'].values))

100%|████████████████████████████████████████████████████████████████████████████| 36086/36086 [53:02<00:00, 11.34it/s]


In [534]:
# inspect a few examples
event_ids[:5]

[['27433105'], ['27433103'], ['27433106'], [], ['27433114']]

Let us check how many relevant matches have Betfair market data.

In [535]:
event_ids_lens = [len(elt) for elt in event_ids]

In [536]:
pd.Series(event_ids_lens).value_counts()

1    32047
0     3956
2       82
3        1
dtype: int64

In [537]:
(pd.Series(event_ids_lens).value_counts().sum() - pd.Series(event_ids_lens).value_counts()[0]) / pd.Series(event_ids_lens).value_counts().sum()

0.8903729978384969

Betfair `event id`s were found for 89% of relevant matches.  
We contacted Betfair and they confirmed that historical data is not available for all matches that actually occured and for which there was a betting market.  
The reasons are technical and may be assumed random.  

Let us investigate the small number of matches with more than one `event id`.

In [538]:
mask_more_than_one_event_id = [True if len(elt) > 1 else False for elt in event_ids]

In [539]:
pd.Series(mask_more_than_one_event_id).value_counts()

False    36003
True        83
dtype: int64

We inspect the data on those matches.  
The DataFrame `more_than_one_event_id` is a subset of Betfair `event_basic_info_df` for those matches.

In [540]:
more_than_one_event_id = pd.DataFrame()

for i in tqdm(range(len(footy[mask_more_than_one_event_id]))):
    mask_country = event_basic_info_df['country_caus'] == footy[mask_more_than_one_event_id].iloc[i]['country_code']
    # bf datetime is TZ aware and footy datetime is not. So explicitly set `.tz_localize(pytz.UTC)`
    mask_datetime = event_basic_info_df['open_date'] == footy[mask_more_than_one_event_id].iloc[i]['date_GMT'].tz_localize(pytz.UTC)
    mask_home = event_basic_info_df['home_team'].isin(team_names_mapping[(footy[mask_more_than_one_event_id].iloc[i]['country_code'], footy[mask_more_than_one_event_id].iloc[i]['home_team_name'])])
    mask_away = event_basic_info_df['away_team'].isin(team_names_mapping[(footy[mask_more_than_one_event_id].iloc[i]['country_code'], footy[mask_more_than_one_event_id].iloc[i]['away_team_name'])])
    mask = mask_country * mask_datetime * mask_home * mask_away
    # event_ids.append(list(event_basic_info_df[mask]['event_id'].values))
    more_than_one_event_id = pd.concat([more_than_one_event_id, event_basic_info_df[mask]])

100%|██████████████████████████████████████████████████████████████████████████████████| 83/83 [00:08<00:00, 10.25it/s]


In [541]:
pd.options.display.max_rows = 200

Let us inspect the DataFrame with data on matches with more than one compatible event id.

In [542]:
more_than_one_event_id

,event_id,country,open_date,home_team,away_team,event_name,country_caus
298208,28970987,AR,2018-10-27 00:00:00+00:00,Huracan,Colon,Huracan v Colon,AR
298363,28962339,AR,2018-10-27 00:00:00+00:00,Huracan,Colon,Huracan v Colon,AR
298217,28972910,AR,2018-10-27 18:00:00+00:00,River Plate,Aldosivi,River Plate v Aldosivi,AR
298364,28962340,AR,2018-10-27 18:00:00+00:00,River Plate,Aldosivi,River Plate v Aldosivi,AR
298216,28972909,AR,2018-10-27 20:15:00+00:00,Gimnasia La Plata,Boca Juniors,Gimnasia La Plata v Boca Juniors,AR
298365,28962342,AR,2018-10-27 20:15:00+00:00,Gimnasia La Plata,Boca Juniors,Gimnasia La Plata v Boca Juniors,AR
277185,28704749,AU,2018-05-05 09:50:00+00:00,Newcastle,Melbourne Victory,Newcastle Jets v Melbourne Victory,AU
277186,28704764,AU,2018-05-05 09:50:00+00:00,Newcastle Jets,Melbourne Victory,Newcastle Jets v Melbourne Victory,AU
295910,28704774,AU,2018-05-05 09:50:00+00:00,Newcastle Jets,Melbourne Victory,Newcastle Jets v Melbourne Victory,AU
297231,28831913,AU,2018-10-19 08:50:00+00:00,Adelaide United,Sydney,Adelaide United v Sydney FC,AU


By inspecting team names, we conclude that even though in some cases there are differences in team names, the names are indeed correct and the plurality of event ids for these matches is not caused by an ambiguity in team names.  
There are also very few cases of more than one name variation for the same `event id` (e.g. `27853333`).  
Let us manually inspect the content of the full Betfair data files for a sample of these matches.

In [543]:
EVENT_ID_INDEX = -2
files_more_than_one_event_id = [file for file in matching_files if file.split('\\')[EVENT_ID_INDEX] in more_than_one_event_id['event_id'].values]

In [544]:
EVENT_ID_INDEX = -2
def get_filenames_for_event_id(event_id, filenames):
    return [file for file in filenames if file.split('\\')[EVENT_ID_INDEX] == event_id]

In [545]:
# display some filenames
for event_id in more_than_one_event_id['event_id'].iloc[:6]:
    print(get_filenames_for_event_id(event_id, files_more_than_one_event_id))

['..\\data\\raw\\betfair\\xds_nfs\\hdfs_supreme\\BASIC\\2018\\Oct\\25\\28970987\\1.150185260.bz2']
['..\\data\\raw\\betfair\\xds_nfs\\hdfs_supreme\\BASIC\\2018\\Oct\\27\\28962339\\1.149749652.bz2']
['..\\data\\raw\\betfair\\xds_nfs\\hdfs_supreme\\BASIC\\2018\\Oct\\25\\28972910\\1.150186213.bz2']
['..\\data\\raw\\betfair\\xds_nfs\\hdfs_supreme\\BASIC\\2018\\Oct\\27\\28962340\\1.149749768.bz2']
['..\\data\\raw\\betfair\\xds_nfs\\hdfs_supreme\\BASIC\\2018\\Oct\\25\\28972909\\1.150186347.bz2']
['..\\data\\raw\\betfair\\xds_nfs\\hdfs_supreme\\BASIC\\2018\\Oct\\27\\28962342\\1.149749894.bz2']


By manually inspecting the content of the full Betfair data files for a sample of matches, we observe that for one of the `event id`s, the data file contains the relevant data, and the data file of the second event id is very limited, without prices, as if the market was set and then cancelled.  
As our objective with Betfair data is to get `in-play` prices, we check **all** data files for an active `in-play` market.  

In [546]:
def check_in_play(bf_data_list):
    """
    Take a list of dictionaries from a Betfair data file and check if the market turns in-play.
    
    Parameter:
    bf_data_list(list): list of dictionaries from a Betfair data file.
    
    Return: Boolean (True if the market turns in-play).
    """
    in_play = False
    for record in bf_data_list:
        if record['mc']:
            for subrecord in record['mc']:
                if 'marketDefinition' in subrecord:
                    if subrecord['marketDefinition']['inPlay'] is True:
                        in_play = True
    return in_play

In [547]:
def get_event_ids_in_play(event_ids, files):
    """
    Take a list of lists of event ids (each list corresponds to a match),
    and return the event ids of matches with active in-play markets.
    
    Parameters:
    event_ids (list of lists): such as [['27433105'], ['27433103'], ['27433106', '27433114'], [], ...]
    files (list): list of filenames, such as ['xds_nfs\\hdfs_supreme\\BASIC\\2018\\Oct\\25\\28970987\\1.150185260.bz2', ...]
    
    Returns:
    event_ids_in_play (list of lists): subset of `event_ids` with only matches with active in-play markets.
    errors_event_ids_in_play (dic): eventual file reading errors.
    """
    event_ids_in_play = []
    errors_event_ids_in_play = {}
    for i, current_event_ids in tqdm(enumerate(event_ids)):
        current_event_ids_in_play = []
        for event_id in current_event_ids:
            # get filenames. Can be more than one per event id.
            files_of_event_id = get_filenames_for_event_id(event_id, files)
            # if in_play in any of files, event_id is relevant.
            for file in files_of_event_id:
                try:
                    bf_data_list = bz2_to_dict_list(file)
                    in_play = check_in_play(bf_data_list)
                    if (in_play and (event_id not in current_event_ids_in_play)):
                        current_event_ids_in_play.append(event_id)
                except Exception as e:
                    errors_event_ids_in_play[file] = e
                    bf_data_list = []
        event_ids_in_play.append(current_event_ids_in_play)
    return event_ids_in_play, errors_event_ids_in_play

In [548]:
event_ids_in_play, errors_event_ids_in_play = get_event_ids_in_play(event_ids, matching_files)

36086it [3:59:14,  2.51it/s]


In [569]:
len(errors_event_ids_in_play)

11

There is a very small number of files with invalid format issues or corrupted.  
Let us check the number of matches with `in-play` market data.

In [549]:
event_ids_in_play_len = [len(elt) for elt in event_ids_in_play]

In [550]:
pd.Series(event_ids_in_play_len).value_counts()

1    31794
0     4292
dtype: int64

In [581]:
pd.Series(event_ids_in_play_len).value_counts().loc[1] / len(event_ids_in_play_len)

0.8810619076650225

As expected, each match has at most one compatible `event id`.  
88% of matches have `in-play` market data.  
Let us flatten the `event_ids_in_play` list of lists and add an `event_id` column to the `footy` DataFrame.

In [551]:
event_ids_in_play_flat = []
for elt in event_ids_in_play:
    if len(elt) == 0:
        event_ids_in_play_flat.append(np.nan)
    elif len(elt) == 1:
        event_ids_in_play_flat.append(elt[0])

In [552]:
footy['event_id'] = event_ids_in_play_flat

In [553]:
footy

,timestamp,date_GMT,status,home_team_name,away_team_name,home_team_goal_count,away_team_goal_count,home_team_goal_timings,away_team_goal_timings,country,country_code,event_id
0,1430589600,2015-05-02 18:00:00,complete,Olimpo,Estudiantes,0,0,NaN,NaN,argentina,AR,27433105
1,1430597700,2015-05-02 20:15:00,complete,Gimnasia La Plata,Newell's Old Boys,0,0,NaN,NaN,argentina,AR,27433103
2,1430601000,2015-05-02 21:10:00,complete,San Lorenzo,Vélez Sarsfield,1,0,83,NaN,argentina,AR,27433106
3,1430605800,2015-05-02 22:30:00,complete,Argentinos Juniors,Aldosivi,0,1,NaN,45'1,argentina,AR,NaN
4,1430608500,2015-05-02 23:15:00,complete,Racing Club,Lanús,2,0,"72,90",NaN,argentina,AR,27433114
...,...,...,...,...,...,...,...,...,...,...,...,...
36081,1583632800,2020-03-08 02:00:00,complete,Colorado Rapids,Orlando City,2,1,"64,90",82,usa,US,29700007
36082,1583636400,2020-03-08 03:00:00,complete,LA Galaxy,Vancouver Whitecaps,0,1,NaN,74,usa,US,29700009
36083,1583636400,2020-03-08 03:00:00,complete,Seattle Sounders,Columbus Crew,1,1,79,33,usa,US,29700008
36084,1583708400,2020-03-08 23:00:00,complete,Portland Timbers,Nashville SC,1,0,12,NaN,usa,US,29713521


As a sanity check, verify that the `event id` per match is unique.

In [586]:
footy['event_id'].dropna().duplicated().any()

False

Export

In [554]:
FOOTY_EVENT_IDS_PATH = '../data/interim/footy_event_ids.csv'
footy.to_csv(FOOTY_EVENT_IDS_PATH, index=False)

## check in-play market data availability per team

Get the `recall` of each team name, in the sense of: (number of relevant matches in footy data set) / (number of matches with in-play market data).  
Outliers per country should be checked for former names and aliases.  
The data below already reflects the result of this iterative process and the last manual checking version.

In [555]:
footy_recall = pd.concat([footy[['country_code', 'home_team_name', 'event_id']].rename(columns={'home_team_name': 'team_name'}),
                          footy[['country_code', 'away_team_name', 'event_id']].rename(columns={'away_team_name': 'team_name'})]).reset_index()

In [556]:
footy_recall['recall'] = footy_recall['event_id'].notna().astype(int)

Group by country and team name.

In [560]:
footy_recall_grouped = footy_recall.groupby(['country_code','team_name']).agg({'recall': ['mean', 'sum', 'count']})

Get totals per country.

In [561]:
footy_recall_grouped_countries = footy_recall.groupby(['country_code']).agg({'recall': ['mean', 'sum', 'count']})
new_cols = list(zip(footy_recall_grouped_countries.index.get_level_values(0), 
                    ['total_country'] * len(footy_recall_grouped_countries.index)))
footy_recall_grouped_countries.index = pd.MultiIndex.from_tuples(new_cols)

In [562]:
footy_recall_grouped_countries_totals = pd.concat([footy_recall_grouped, footy_recall_grouped_countries]).sort_index()

Flatten column names, so we can use `style.format` easily.

In [563]:
footy_recall_grouped_countries_totals.columns = [' '.join(col).strip() for col in footy_recall_grouped_countries_totals.columns.values]

Set `pandas` to display all rows, so we can inspect all team names.

In [558]:
pd.set_option('display.max_rows', None)

In [564]:
footy_recall_grouped_countries_totals.style.format(precision=2).background_gradient(subset='recall mean')

Team names with low-ish `in-play data-wise recalls` were checked.  
In some cases, former names and aliases were found.  
The remaining team names with low-ish `in-play data-wise recalls` are from less traditional leagues and/or have less matches.

In [590]:
RECALL_EXAMPLE_THRESHOLD = .65
footy_recall_grouped_countries_totals[
    footy_recall_grouped_countries_totals['recall mean'] < RECALL_EXAMPLE_THRESHOLD
].style.format(precision=2).background_gradient(subset='recall mean')

The team names may be considered validated and the relevant `event id`s have been fetched.